In [34]:
import json
from datetime import datetime
resFile = 'results/april17/experiments100.rest.res.json'

In [35]:
"""
Check which workers completed the golden HITs correctly.
"""
import csv

workers = {}
with open(resFile) as f:
    for line in f:
        jObj = json.loads(line.strip())
        worker = jObj['workerID']
        if worker not in workers:
            workers[worker] = []
        workers[worker].append(jObj)


for w in workers:
    choseCorrectly = True
    i = 0
    
    print('***')
    for j in workers[w]:
        i += 1
        if j['golden']:
            
            choseCorrectly = (choseCorrectly and (j['selected'] == j['bestanswer']))
            if not choseCorrectly:
                print('%d :: FALSE:: Worker :: %s, golden correct: %s :: expID :: %s' % (i, w, j['selected'] == j['bestanswer'], j['expID']))
                
#                 print("expID :: %s" % j['expID'])
                print(datetime.fromtimestamp(int(j['timestamp']['$numberLong'])/1000))
                print('{left} - {right}'.format(left=j['left'], right=j['right']))
                print("choseCorrectly :: %s\n" % choseCorrectly)
                
#             else:
#                 print('%d :: Worker :: %s, golden correct: %s :: expID :: %s' % (i, w, choseCorrectly, j['expID']))


***
***


In [36]:
"""
Get IDs of completed experiments.
"""

allJudgements = []
expIDs = {}

for w in workers:
    for j in workers[w]:
        allJudgements.append(j)

for j in allJudgements:
    if j['expID'] in expIDs:
        expIDs[j['expID']].add(j['workerID'])
    else:
        expIDs[j['expID']] = set()
        expIDs[j['expID']].add(j['workerID'])
               

for e in expIDs.items():
    print(e[0])
    print(e[1])
    print('***')

35EF362B2F991121
{'A1OUUZSD5DD4ZN'}
***
6F96A1BC4C98E5B2
{'A1OUUZSD5DD4ZN'}
***
E85E95ADABBF366B
{'A1PTH9KTRO06EG'}
***


In [35]:
import csv
def loadExpIDs(filename):
    ids = []
    with open(filename) as f:
        reader = csv.DictReader(f, fieldnames=['EXPERIMENT_ID'])
        next(reader)
        for line in reader:
            ids.append(line['EXPERIMENT_ID'])
    return ids

In [6]:
import csv, json
def loadResults(filename):
    res = []
    with open(filename) as f:
        for line in f:
            jObj = json.loads(line.strip())
            if type(jObj['timestamp']) != type(1):
                t = jObj['timestamp']['$numberLong']
                jObj['timestamp'] = int(t)
            res.append(jObj)
    return res

In [37]:
# expIDs = loadExpIDs('experiments000_batch1.csv')
res = loadResults('results/april17/experiments100.rest.res.json')

filteredResults = []
for r in res:
    if r['golden']:
        continue
    filteredResults.append(r)

fOut = 'results/april17/experiments100.rest.res.no_golden.json'
with open(fOut, 'w') as f:
    for r in filteredResults:
        f.write(json.dumps(r))
        f.write('\n')


In [8]:
def loadPairs(fname):
    pairs = []
    with open(fname) as f:
        for line in f:
            topic, c1, c2 = line.strip().split()
            pairs.append( (topic, c1, c2) )
    return pairs

In [60]:
# fOut = 'results/experiments000_results_no_golden.json'
with open(fOut, 'w') as f:
    for r in res:
        if r['golden']:
            continue
        f.write(json.dumps(r))
        f.write('\n')

In [38]:
from datetime import datetime
res = loadResults('results/april17/test.100.res.-30.json')

d = {}
i = 0
for r in res:
    left = r['left']
    right = r['right']
    selected = r['selected']
    expID = r['expID']
    workerID = r['workerID']
    t = r['timestamp']
    p = (left, right, expID, workerID)
    if p in d:
        i += 1
        d[p].append( (selected, t) )
    else:
        d[p] = [ (selected, t) ]
        
for i in d.items():
    if len(i[1]) > 1:
        print(i[0])
#         print(i[1])
        for c in i[1]:
            dt = datetime.fromtimestamp(c[1]/1000)
            print('{ts} :: {time} :: {choice}'.format(ts=c[1], time=dt, choice=c[0]) )
        print('*'*20)
        
#         1545730073
#         1584567872796

In [26]:
print(len(pairs))
print(len(res))

3498
1987


In [39]:
res = loadResults('results/april17/test.100.res.-30.json')
pairs = loadPairs('results/april17/test.100')
myselections = []
i = 0
counter = 0
print(len(res))
notjudged = []
judged = []
for p in pairs:
    foundAnswer = False
    i += 1
    topic, c1, c2 = p
    j = 0
    used = None

    for r in res:
        assert(r['left'] != r['right'])
        if ((r['topic'] == topic) and (r['left'] in [c1, c2]) and (r['right'] in [c1, c2])):
#             j += 1
#             if j > 1:
#                 print(j)
#                 print(i)
#                 print(topic, c1, c2)
#                 print(r)
#                 print('*'*20)
            myselections.append((topic, c1, c2, r['selected']) )
            used = r
            foundAnswer = True
            break
        
    if not foundAnswer:
        notjudged.append(p)
    else:
        judged.append(p)
    try:
        res.remove(used)
    except ValueError as e:
        counter += 1
#         print(used)
    
print('****')
# print(len(res))            
# print(i)
# print(counter)
# print(len(myselections))
print('total pairs: {l}'.format(l=len(pairs)))
print('unjudged pairs: {l}'.format(l=len(notjudged)))
print('judged pairs: {l}'.format(l=len(judged)))


#             

3518
****
total pairs: 3498
unjudged pairs: 0
judged pairs: 3498


In [32]:
for p in notjudged:
    print('{topic} {left} {right}'.format(topic=p[0], left=p[1], right=p[2]))

32_10 CAR_c9e04e949d7107321e263508cff30eb98a13a560 MARCO_6763412
32_11 MARCO_3632497 MARCO_7523018
33_6 MARCO_7273395 MARCO_7273403
34_2 CAR_1b19bac71cf0147b06d28a908c3ee204d1ac3839 CAR_cded43587fd0308695dad1d0d9a1c1d4a57528bf
34_4 CAR_a8be49221a232cb39a76943ba0c44990203fb1ec CAR_a8e08171094026d8625d619a61d07583f4ba255b
34_6 CAR_84b09d0fc9068e7332b8af9d8ecf1cec7ff053b4 CAR_f5103b66a05d8c54f56b676cc5363d4ad9ff937e
37_7 CAR_eb7fe94c6784df26dfcdf5aadcc656f043ee2c75 MARCO_8834764
40_5 CAR_224fe1e4b78faf97a1319f1b98f2c2e0446bd9e4 CAR_dce685d14bfa5a44eac27212fe432d9436ad823f
50_3 CAR_16eccf2a08f153ab8136484d2bc3125bdc226bff CAR_3e82b449536bab5c5cf63ef3366d7af62d585497
54_3 CAR_f31c92aada52df5b0f60a8e3b7f24d7c586bd8f5 MARCO_620464
54_6 MARCO_6422474 MARCO_6475243
58_2 CAR_0c566fee99742c8af35c2d631a94d47b9a5764a8 CAR_148c1be6a4c6b2a2eefa0379b608dd25b0290838
59_5 MARCO_1426190 MARCO_8757626
69_7 MARCO_5932552 MARCO_5932554
75_5 MARCO_1478539 MARCO_1628228
31_2 MARCO_4976184 MARCO_8046970
50_8 C

In [40]:
with open('results/april17/test.100.res.full', 'w') as f:
    for s in myselections:
        f.write('{topic} {c1} {c2} {ans}\n'.format(topic=s[0], c1=s[1], c2=s[2], ans=s[3]))

In [16]:
notJudged = []
for p in pairs:
    if p not in judged:
        notJudged.append(p)
        
for p in notJudged:
    print(p)
        

NameError: name 'judged' is not defined

In [17]:
for p in notjudged:
    print(p)
#expID     6D6184E2E127A6EA

In [102]:
print(len(res))

0


In [98]:
print(len(myselections))

2502


In [95]:
a  = 1
b = 2

c = 2
d = 1

print(((a,b) == (c,d)) or ((b,a) == (c,d)))
print((a in (c,d)) and (b in (c,d)))

True
True


In [35]:
res = loadResults('results/march26/experiments001_res_no_golden.json')
workerDict = {}
for r in res:
    w = r['workerID']
    if w not in workerDict:
        workerDict[w] = set()
        workerDict[w].add(r['expID'])
    else:
        workerDict[w].add(r['expID'])
#     workerDict[w] = workerDict.get(w, 0) + 1
    
for i in workerDict.items():
    print('{w}: {c}\n'.format(w=i[0], c=i[1]))

AK4WZEW584BR9: {'7DAD5753CB59D800'}

A2P065E9CYMYJL: {'21B80A1797FF9D35'}

A1F669OTXWIJW0: {'ECE9D5AD3D7A4F97'}

A2JP9IKRHNLRPI: {'A22DBD622AE773D0', 'BD0C5329074E547C', 'F9FF2DB24CBB0A3F'}

A14EYTLSMJRPUK: {'BD0C5329074E547C'}

A1CSL2I9M8H1FI: {'5EFF5E5F1FE47C08'}

A35D31QHYQUF9V: {'6921A4D9C08B579F'}

A37OUZOGQKGMW0: {'41E986B4D30A9922'}

A1SX8IVV82M0LW: {'92D52CEC16D69AC3', 'AF8ED1E4B3B883A1', 'C11C778706F500F6', '6D6184E2E127A6EA', 'A22DBD622AE773D0', '1CE9B15520DCFB71', '1AFB4A15AD563FF1'}

A26LOVXF4QZZCO: {'92D52CEC16D69AC3'}

A10249252O9I20MRSOBVF: {'E471310CC7271F81'}

A1E77HZO63E334: {'4830613628355D38', '897826E128391D75', '92D52CEC16D69AC3', '41E986B4D30A9922', '843F34FA01EFF378', 'B93FA5F8E1287E9A', '1AFB4A15AD563FF1'}

A30GJ0HHLO16E8: {'E0B23401F8A09CF8'}

A37S96RT1P1IT2: {'AE54F7EE9C726A37'}

A27SMEOPKV84VI: {'A19BCB691076FF8F'}

A1P2RQ166VS5BT: {'408929A5C7CF09B9'}

A3UUH3632AI3ZX: {'B93FA5F8E1287E9A'}

A3J85WP15JFYW0: {'B93FA5F8E1287E9A'}

A18G2CLYSTENK: {'990BA8360D742